In [1]:
#XML parser for Canadian Record of Parliamentary Debates (Senate)
#version: M. Wong, 2018-01-18

######################################################################################################################
# OVERVIEW
######################################################################################################################
#Notes:     (1) Python 3.6 Anaconda
#           (2) Encoding UTF-8
#           (3) Runs from parent directory of data folder.
#               Run this file from the folder that contains the html data folder
#               downloaded from www.sencanada.ca [TO DO: Add and Link]

## Initialization

In [2]:
#Load Modules
import os
import re
import pandas as pd
import numpy as np
import math
import time
import spacy
from datetime import datetime
from bs4 import BeautifulSoup
from spacy.lang.en import English

#Set pandas to display full column widths and full column numbers
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

#Spacy Parser
nlp = spacy.load('en')

In [3]:
#Load Authority File as pandas dataframe
authorityFile_df = pd.read_json("authorityFile/authorityFile_senate.json",orient="columns")
#authorityFile_df

#Get new name column in firstnamelastname format for parsing
authorityFile_df = authorityFile_df.sort_index()
#authorityFile_df

In [4]:
#Open example final speech csv from House
House_final_csv = pd.read_csv("hansardExtractedSpeechesFull.csv", delimiter="\t", nrows=20, index_col=0)
#House_final_csv

In [5]:
## Copy format of House final csv for Senate csv
Senate_final_csv = House_final_csv.head(0).copy()
#display(Senate_final_csv)

#columns_list = list(Senate_final_csv.columns.values)
#print(columns_list)

## Function Declarations

In [6]:
## These functions specifies wanted tags within other tags (ex. <a> within <h1> or <b> within <p>). Not children
## Returns two lists, one of ids and one of lines, denoted by the inside tag that reside in the outside tag (given attributes)
def tag_specifier(soup, outside_tag, inside_tag, class_attr_outside, id_attr_outside, class_attr_inside, id_attr_inside):
    return [a[0].get("id") for a in (td.find_all(inside_tag, class_=class_attr_outside, id=id_attr_inside) for td in soup.find_all(outside_tag, class_=class_attr_outside, id=id_attr_outside)) if a], [a[0] for a in (td.find_all(inside_tag, class_=class_attr_outside, id=id_attr_inside) for td in soup.find_all(outside_tag, class_=class_attr_outside, id=id_attr_outside)) if a]

def tag_specifier_simple(soup, outside_tag, inside_tag):
    return [a[0].get("id") for a in (td.find_all(inside_tag) for td in soup.find_all(outside_tag)) if a], [a[0] for a in (td.find_all(inside_tag) for td in soup.find_all(outside_tag)) if a]

def tag_specifier_textonly(soup, outside_tag, inside_tag, class_attr_outside, id_attr_outside, class_attr_inside, id_attr_inside):
    return [a[0].get("id") for a in (td.find_all(inside_tag, class_=class_attr_outside, id=id_attr_inside) for td in soup.find_all(outside_tag, class_=class_attr_outside, id=id_attr_outside)) if a], [a[0].get_text() for a in (td.find_all(inside_tag, class_=class_attr_outside, id=id_attr_inside) for td in soup.find_all(outside_tag, class_=class_attr_outside, id=id_attr_outside)) if a]



def authorityFile_get_ID(firstname, lastname, column):
    index = authorityFile_df.loc[(authorityFile_df["firstName"] == firstname) & (authorityFile_df["lastName"] == lastname)].index[0]
    value = authorityFile_df[column][index]
    if value == "" or value == None:
        return "NA"
    else:
        return value

def authorityFile_get(speakerName, name_id_dict):
    for lastname in authorityFile_df["lastName"]:
        #print(lastname)
        if lastname in speakerName:
            for firstname in authorityFile_df.loc[authorityFile_df["lastName"] == lastname]["firstName"]:
                #print(firstname)
                if firstname in speakerName:
                    #print(firstname, lastname)
                    ## Get relevant info from authorityFile entry given name
                    
                    parlInfoId = int(authorityFile_get_ID(firstname, lastname, "parlInfoId"))
                    fullName = authorityFile_df[authorityFile_df["parlInfoId"] == parlInfoId]["fullName"].values[0]
                    firstName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["firstName"].values[0]
                    lastName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["lastName"].values[0]
                    middleName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["middleName"].values[0]
                    sex = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["sex"].values[0]
                    dateOfBirth = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["dateOfBirth"].values[0]    
                    isEstimateDOB = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["isEstimateDOB"].values[0]
                    visibleMinority = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["visibleMinority"].values[0]
                    indigenous = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["indigenous"].values[0]
                    birthProvince = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["birthProvince"].values[0]
                    birthCountry = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["birthCountry"].values[0]
                    firstDay = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["firstDay"].values[0]
                    provOfRiding = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["provOfRiding"].values[0]
                    parlInfoPage = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["parlInfoPage"].values[0]
                    daysInOffice = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["daysInOffice"].values[0]

                    name_id_dict[lastName] = parlInfoId
                    
                    return (parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, 
                            visibleMinority, indigenous, birthProvince, birthCountry, firstDay, provOfRiding, 
                            parlInfoPage, daysInOffice, name_id_dict)
    
    else:
        parlInfoId = None
        fullName = None
        firstName = None
        lastName = None
        middleName = None
        sex = None
        age = None
        dateOfBirth = None
        isEstimateDOB = None
        visibleMinority = None
        indigenous = None
        birthProvince = None
        birthCountry = None
        firstDay = None
        provOfRiding = None
        parlInfoPage = None
        daysInOffice = None
        
        return (parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, visibleMinority,
                indigenous, birthProvince, birthCountry, firstDay, provOfRiding, parlInfoPage, daysInOffice, 
                name_id_dict)
    

def speakerName_check(speakerName, name_id_dict, current_speaker):
    #speakerName = speakerName.replace(":","")

    #print(speakerName)                         

    ####### Check last names of authorityFile and see if they match name, and then match first names if 
    ####### multiple last names are found

    ## Speaker (the Senate position) designation converted to current senate Speaker name
    if "Hon. the Speaker" in speakerName:
        speakerName = current_speaker[0].get_text()                        
        parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, visibleMinority, indigenous, birthProvince, birthCountry, firstDay, provOfRiding, parlInfoPage, daysInOffice, name_id_dict = authorityFile_get(speakerName,name_id_dict)

    ## Account for some hansard formatting rules
    ## When already introduced, subsequent references sometimes refer to "Senator lastname" 
    ## Therefore, must record introduced senators with dict of parlInfoId and last name
    elif "Senator " in speakerName:
        #print(name_id_dict)
        try:
            parlInfoId = int(name_id_dict[speakerName.split(" ")[1]])
            fullName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["fullName"].values[0]
            firstName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["firstName"].values[0]
            lastName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["lastName"].values[0]
            middleName = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["middleName"].values[0]
            sex = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["sex"].values[0]
            dateOfBirth = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["dateOfBirth"].values[0]
            isEstimateDOB = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["isEstimateDOB"].values[0]
            visibleMinority = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["visibleMinority"].values[0]
            indigenous = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["indigenous"].values[0]
            birthProvince = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["birthProvince"].values[0]
            birthCountry = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["birthCountry"].values[0]
            firstDay = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["firstDay"].values[0]
            provOfRiding = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["provOfRiding"].values[0]
            parlInfoPage = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["parlInfoPage"].values[0]
            daysInOffice = authorityFile_df.loc[authorityFile_df["parlInfoId"] == parlInfoId]["daysInOffice"].values[0]
        except:
            parlInfoId = None
            fullName = None
            firstName = None
            lastName = None
            middleName = None
            sex = None
            dateOfBirth = None
            isEstimateDOB = None
            visibleMinority = None
            indigenous = None
            birthProvince = None
            birthCountry = None
            firstDay = None
            provOfRiding = None
            parlInfoPage = None
            daysInOffice = None
            
    else:
        parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, visibleMinority, indigenous, birthProvince, birthCountry, firstDay, provOfRiding, parlInfoPage, daysInOffice, name_id_dict = authorityFile_get(speakerName,name_id_dict)                                            

    return (parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, visibleMinority,
            indigenous, birthProvince, birthCountry, firstDay, provOfRiding, parlInfoPage, daysInOffice, 
            name_id_dict)

## BeautifulSoup Parser

In [7]:
#Set list of available parliaments and respective sessions
parliamentNumber_list = [42]#[42,41,40,39,38,37,36,35]
parliamentSession_list = [1]#[3,2,1]

In [16]:
#Main parser function definition
def senate_hansard_parser(parliamentNumber,parliamentSession,Senate_final_csv):
    print("Senate Hansard Parliament "+str(parliamentNumber)+" Session "+str(parliamentSession))
    
    #Use beautifulsoup to parse HTML
    
    try:
        directory = "Data/Hansard/forParser/Senate Hansard Parliament "+str(parliamentNumber)+" Session "+str(parliamentSession)
        
        ## List of lists, which will be appended to final csv
        ## Each sublist is a row in the final csv
        ## Each list encompasses all speeches in a single HTML file (aka in one sitting)
        
        for i,html_file in enumerate(os.listdir(directory)):
            all_rows = []
            filename = os.fsdecode(html_file)
            print("HTML: "+os.path.join(directory, filename))
            print("")
            
            ## Get last known interventionId from file
            id_filename = str(parliamentNumber)+str(parliamentSession)+"_previous_interventionId"
            try:
                with open(os.path.join(directory,id_filename)+".txt", "r") as id_file:
                    previous_interventionId = int(id_file.read())
                    #print(previous_interventionId)
            except FileNotFoundError:
                previous_interventionId = 0
            
            #Get date from filename
            dateYMD = (filename.split("_")[2]).split(".")[0]
            #print(dateYMD)
            year, month, day = int(dateYMD.split("-")[0]), int(dateYMD.split("-")[1]), int(dateYMD.split("-")[2])
            #print(year,month,day)
            
            date = datetime(int(year), int(month), int(day))
            date = date.strftime("%B %d %Y")  
            #print(date)
            
            orderOfBusinessRubric_list = []
            subjectOfBusinessTitle_list = []
            with open(os.path.join(directory, filename)) as file:
                soup = BeautifulSoup(file,"html.parser")
                
                
                ## IMPORTANT: Because beautifulsoup does not have line number extraction and Senate hansard HTMLs
                ## do not have a nice format, must do everything manually
                
                ## Get entire HTML as list of lines for parsing
                #print(soup.prettify())
                all_lines = soup.find_all()
                #print(len(all_lines))
                #print(all_lines)
            
                #NOTE: orders of business and subject of business do not have class attributes and are denoted by ids in the <a> tag
                orderOfBusinessRubric_id_list, orderOfBusinessRubric_list = tag_specifier(soup,"h1","a",None,None,None,True)
                #print(orderOfBusinessRubric_id_list)
                #print(orderOfBusinessRubric_list)
                #print("")
                
                subjectOfBusinessTitle_id_list, subjectOfBusinessTitle_list = tag_specifier(soup,"h2","a",None,None,None,True) 
                #print(subjectOfBusinessTitle_id_list)
                #print(subjectOfBusinessTitle_list)
                #print("")
                
                ## Get name of current Senate speaker 
                current_speaker = tag_specifier_simple(soup,"h2","span")[1]
                #print(current_speaker)
                #print("")
                
                ## Get subject of business qualifiers
                subjectOfBusinessQualifier_list = tag_specifier_simple(soup,"h3","a")[1]
                #print(subjectOfBusinessQualifier_list)
                #print("")                
                
                ## Get all italicized lines
                #italicized_list = soup.find_all("i", class_=None, align=None)
                #print(italicized_list)
                #print("")
                
                ## Get all speeches (denoted by <p> with no class)
                speech_list = soup.find_all("p", class_=None, align=None)
                #print(speech_list)
                #print("")
                
                ## Get all speaker names (denoted by <p> with <b> inner tag), avoid those within <u> tags
                speakerName_unfiltered_list = tag_specifier_textonly(soup,"p","b",None,None,None,None)[1]
                speakerName_u_list = tag_specifier_textonly(soup,"p","u",None,None,None,None)[1]
                speakerName_list = [name for name in speakerName_unfiltered_list if name not in speakerName_u_list]
                #print(speakerName_list)
                #print("")
                
                #Set default values
                orderOfBusinessRubric = None
                subjectOfBusinessID = None
                subjectOfBusinessTitle = None
                subjectOfBusinessQualifier = None
                speechId = None
                interventionId = None
                weekday = None
                timeStamp = None
                speakerName = None
                #Party seems difficult to retrieve
                party = None
                speechFiltered = ""
                floorLanguage = "EN"
                
                parlInfoId = None
                fullName = None
                firstName = None
                lastName = None
                middleName = None
                sex = None
                age = None
                dateOfBirth = None
                isEstimateDOB = None
                visibleMinority = None
                indigenous = None
                birthProvince = None
                birthCountry = None
                firstDay = None
                provOfRiding = None
                parlInfoPage = None
                daysInOffice = None
                
                #Not sure how to get mentioned entities, so ignore again for now
                affiliationType = None
                affiliationDbId = None
                mentionedDocumentsTitle = None
                mentionedDocumentsId = None
                mentionedDocumentsType = None
                mentionedEntityName = None
                mentionedEntityId = None
                mentionedEntityType = None
                
                name_id_dict = dict()               
                
                for line in all_lines:
                    #print(line)
                    ## Order of business, subject of business and speaker names are renewed only when a valid line is encountered
                    ## Otherwise, these values are kept at the previous values
                    
                    if line in orderOfBusinessRubric_list:
                        #print(orderOfBusinessRubric_id_list[orderOfBusinessRubric_list.index(line)])
                        
                        orderOfBusinessRubric = line.get_text()
                        #print(orderOfBusinessRubric) 
                        
                    elif line in subjectOfBusinessTitle_list:
                        subjectOfBusinessID = subjectOfBusinessTitle_id_list[subjectOfBusinessTitle_list.index(line)]
                        #print(subjectOfBusinessID)
                        
                        subjectOfBusinessTitle = line.get_text()
                        #print(subjectOfBusinessTitle)
                        
                        ## Reset qualifier to default
                        subjectOfBusinessQualifier = None
                        
                    elif line in subjectOfBusinessQualifier_list:
                        subjectOfBusinessQualifier = line.get_text()
                        #print(subjectOfBusinessQualifier)
                    
                    #elif line in speakerName_filtered_list:
                    #    print(i, line)
                    #    speakerName = line.get_text()
                        #print(i,speakerName)
    
                    elif line in speech_list:
                        #speech = str(line)
                        speech = line.get_text()
                        #print(speech)
                        
                        if speech == "":
                            pass
                    
                        elif (speakerName == None) and not (speech.startswith("Hon. ")):
                            pass
                        
                        elif speech.startswith("Hon. Senators"):
                            #Do not add to dataframe if there are multiple speakers
                            pass
                        
                        elif "On the Order" in speech:
                            pass

                        ## Detect language changes and other signifiers
                        #elif any(speech in s for s in italicized_list):
                        #    print(speech)
                            ## Attach response identification to qualifiers if applicable
                        elif "(Response to question raised" in speech:
                            try:
                                subjectOfBusinessQualifier = subjectOfBusinessQualifier + " " + speech
                                #print(subjectOfBusinessQualifier)
                                #print("")
                            except Exception:
                                pass
                        elif "[Translation]" == speech:
                            floorLanguage = "FR"
                            #print(floorLanguage)
                            #print("")
                        elif "[English]" == speech:
                            floorLanguage = "EN"
                            #print(floorLanguage)
                            #print("")
                        
                        ## Get speakerNames
                        elif speech.startswith("Hon. ") or speech.startswith("The Hon. ") or speech.startswith("Senator "):
                        ## Collect all info as a series of lists to then append to the final dataframe
                        
                            ## Catch all (most?) potential ways of presenting speakerNames and speeches    
                            
                            if (")" in speech) and ("," in speech) and (":" in speech):
                                ## Split on ) if it is before , or :, keep ) at end of string if true
                                index_bracket, index_comma, index_colon = speech.index(")"), speech.index(","), speech.index(":")
                                #print(index_bracket, index_colon)
                                if (index_bracket < index_comma) and (index_bracket < index_colon):
                                    try:
                                        speakerName, speech = speech.split(")",1)[0], speech.split(": ",1)[1]
                                        speakerName += ")"
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(")",1)[0]
                                        speakerName += ")"
                                        speech = ""
                                elif (index_comma < index_bracket) and (index_comma < index_colon):
                                    try:
                                        speakerName, speech = speech.split(", ",1)[0], speech.split(": ",1)[1]
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(", ",1)[0]
                                        speech = ""
                                else:
                                    try:
                                        speakerName, speech = speech.split(": ",1)[0], speech.split(": ",1)[1]
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(":",1)[0]
                                        speech = ""                                    

                            elif (")" in speech) and (":" in speech):
                                index_bracket, index_colon = speech.index(")"), speech.index(":")
                                if index_bracket < index_colon:
                                    try:
                                        speakerName, speech = speech.split(")",1)[0], speech.split(": ",1)[1]
                                        speakerName += ")"
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(")",1)[0]
                                        speakerName += ")"
                                        speech = ""
                                else:
                                    try:
                                        speakerName, speech = speech.split(": ",1)[0], speech.split(": ",1)[1]
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(":",1)[0]
                                        speech = ""
                                
                            elif ("," in speech) and (":" in speech):
                                index_comma, index_colon = speech.index(","), speech.index(":")
                                if index_comma < index_colon:
                                    try:
                                        speakerName, speech = speech.split(", ",1)[0], speech.split(": ",1)[1]
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(", ",1)[0]
                                        speech = ""
                                else:
                                    try:
                                        speakerName, speech = speech.split(": ",1)[0], speech.split(": ",1)[1]
                                    except IndexError:
                                        ## In instances where no speech follows speakerName
                                        speakerName = speech.split(":",1)[0]
                                        speech = ""        
                            
                            elif (":" not in speech) and ("introduced Bill" in speech):
                                try:
                                    subjectOfBusinessQualifier = subjectOfBusinessQualifier + " " + speech
                                except TypeError:
                                    subjectOfBusinessQualifier = speech
                            
                            elif (":" not in speech) and ("introduced Bill" not in speech):
                                pass
                                    
                            else:
                                try:
                                    speakerName, speech = speech.split(": ",1)[0], speech.split(": ",1)[1]
                                except IndexError:
                                    ## In instances where no speech follows speakerName
                                    speakerName = speech.split(":",1)[0]
                                    speech = ""
                            
                            if any(speakerName in s for s in speakerName_list):
                                #print(speakerName)
                                parlInfoId, fullName, firstName, lastName, middleName, sex, dateOfBirth, isEstimateDOB, visibleMinority, indigenous, birthProvince, birthCountry, firstDay, provOfRiding, parlInfoPage, daysInOffice, name_id_dict = speakerName_check(speakerName, name_id_dict, current_speaker)
                            else:
                                pass
                            
                            # ['parliamentNumber', 'parliamentSession', 'orderOfBusinessRubric', 'subjectOfBusinessTitle', 
                            # 'subjectOfBusinessID', 'subjectOfBusinessQualifier', 'speechId', 'interventionId', 'date', 
                            # 'dateYMD', 'year', 'month', 'day', 'weekday', 'timeStamp', 'speakerName', 'party', 
                            # 'parlInfoId', 'fullName', 'firstName', 'lastName', 'middleName', 'sex', 'age', 'daysInOffice',
                            # 'visibleMinority', 'indigenous', 'dateOfBirth', 'isEstimateDOB', 'birthProvince', 
                            # 'birthCountry', 'firstDay', 'provOfRiding', 'parlInfoPage', 'affiliationType', 
                            # 'affiliationDbId', 'floorLanguage', 'speech', 'speechFiltered', 'mentionedDocumentsTitle', 
                            # 'mentionedDocumentsId', 'mentionedDocumentsType', 'mentionedEntityName', 'mentionedEntityId',
                            # 'mentionedEntityType', 'filename']
                            
                            #print(speech)
                            speech_nlp = nlp(speech) # converting speech into Spacy doc

                            for token in speech_nlp:
                                pair = "_".join([token.text, token.tag_])
                                speechFiltered = " ".join([speechFiltered, pair])

                            row = [parliamentNumber, parliamentSession, orderOfBusinessRubric, subjectOfBusinessTitle,
                                      subjectOfBusinessID, subjectOfBusinessQualifier, speechId, interventionId, date, 
                                      dateYMD, year, month, day, weekday, timeStamp, speakerName, party, parlInfoId,
                                      fullName, firstName, lastName, middleName, sex, age, daysInOffice, visibleMinority,
                                      indigenous, dateOfBirth, isEstimateDOB, birthProvince, birthCountry, firstDay, provOfRiding,
                                      parlInfoPage, affiliationType, affiliationDbId, floorLanguage, speech,
                                      speechFiltered, mentionedDocumentsTitle, mentionedDocumentsId, mentionedDocumentsType, 
                                      mentionedEntityName, mentionedEntityId, mentionedEntityType, filename]

                            ## Reset filtered speech to default
                            speechFiltered = ""

                            #print(df_row)
                            #print(len(df_row))

                            all_rows.append(row)
                            del row
                        
                        else:
                            #print(speech)
                            speech_nlp = nlp(speech) # converting speech into Spacy doc

                            for token in speech_nlp:
                                pair = "_".join([token.text, token.tag_])
                                speechFiltered = " ".join([speechFiltered, pair])

                            row = [parliamentNumber, parliamentSession, orderOfBusinessRubric, subjectOfBusinessTitle,
                                      subjectOfBusinessID, subjectOfBusinessQualifier, speechId, interventionId, date, 
                                      dateYMD, year, month, day, weekday, timeStamp, speakerName, party, parlInfoId,
                                      fullName, firstName, lastName, middleName, sex, age, daysInOffice, visibleMinority,
                                      indigenous, dateOfBirth, isEstimateDOB, birthProvince, birthCountry, firstDay, provOfRiding,
                                      parlInfoPage, affiliationType, affiliationDbId, floorLanguage, speech,
                                      speechFiltered, mentionedDocumentsTitle, mentionedDocumentsId, mentionedDocumentsType, 
                                      mentionedEntityName, mentionedEntityId, mentionedEntityType, filename]

                            ## Reset filtered speech to default
                            speechFiltered = ""

                            all_rows.append(row)
                            del row

            
            ## Once the HTML file is parsed, append to csv            
            ## Create new dataframe that aggregates consecutive speeches of the same speaker in the same language 
            ## and in the same subjectOfBusinessTitle/subjectOfBusinessQualifier
            refined_all_rows, row_previous = [], all_rows[0]
            for row in all_rows[1:]:
                
                ## fullName, floorLanguage, subjectOfBusinessTitle and subjectOfBusinessQualifier
                ## are index 18, 36, 3 and 5 respectively
                
                ## If row changes in those areas, append previous row to new frame
                if (row_previous[18] != row[18]) or (row_previous[36] != row[36]) or (row_previous[3] != row[3]) or (row_previous[5] != row[5]):
                    refined_all_rows.append(row_previous)
                    row_previous = row
                    
                ## Else, keep all column values the same as previous row but append speech and speechFiltered
                ## speech and speechFiltered are index 37 and 38 respectively
                else:
                    #print(row_previous)
                    row_previous[37] = row_previous[37] + " " + row[37]
                    row_previous[38] = row_previous[38] + " " + row[38] 
                    #print(row_previous)
            
            ## Add in interventionId and speechId
            ## interventionId is formatted as a 7-digit number
            ## speechId is interventionId appended to date of speech
            ## speechId is index 6, interventionId is index 7, dateYMD is index 9
            
            for row in refined_all_rows:
                previous_interventionId += 1
                #interventionId
                row[7] = str(previous_interventionId).zfill(7)
                #speechId
                row[6] = str(row[9])+"-"+str(row[7])
            
            #print(refined_all_rows)
            
            df_refined_all_rows = pd.DataFrame(refined_all_rows, columns=Senate_final_csv.columns)
            #display(df_refined_all_rows)
            del refined_all_rows
            
            ## Record dataframe to csv
            if i > 0:
                with open("hansardExtractedSpeechesFull_senate.csv","a") as file:
                    df_refined_all_rows.to_csv(file,header=False,index=False)
            elif i == 0:
                df_refined_all_rows.to_csv("hansardExtractedSpeechesFull_senate.csv",index=False,header=True)
                
            del df_refined_all_rows
            
            ## Record final interventionId to file
            with open(os.path.join(directory,id_filename)+".txt", "w") as id_file:
                id_file.write(str(previous_interventionId))
            
            #break
            
        #display(Senate_final_csv.head(n=20))
    
    #Display error if parliament-session combo does not exist
    except FileNotFoundError:
        print("Error: Parliament and session combination does not exist")
        print("")

## Function Call

In [17]:
for parliamentNumber in parliamentNumber_list:
    for parliamentSession in parliamentSession_list:
        senate_hansard_parser(parliamentNumber,parliamentSession,Senate_final_csv)

Senate Hansard Parliament 42 Session 1
HTML: Data/Hansard/forParser/Senate Hansard Parliament 42 Session 1/421_0001_2015-12-03.html

<p>Parliament having been summoned by proclamation to meet this day for the 
dispatch of business:</p>
<p><b>The Honourable George J. Furey,</b> having taken the Clerk's chair, rose 
and informed the Senate that a Commission had been issued under the Great Seal 
of Canada, appointing him Speaker of the Senate.</p>
<p><i>The said Commission was then read by the Clerk.</i></p>
<p><b>The Hon. the Speaker</b> then took the chair at the foot of the Throne, to 
which he was conducted by the Honourable Claude Carignan, P.C., and the 
Honourable James S. Cowan, the Usher of the Black Rod preceding.</p>
<p>Prayers.</p>
<p><b>The Hon. the Speaker:</b> Thank you, colleagues; thank you for such a warm 
welcome.</p>
<p>Honourable senators, our esteemed former Speaker, the Honourable Pierre 
Claude Nolin, began to speak very briefly to this chamber before he took his 


<p>The Senate met at 2:00 p.m., the Speaker in the chair.</p>
<p>Prayers.</p>
<p>[<i>Translation</i>]</p>
<p></p>
<p>It is a bit odd to be on this side of the chamber, but it has its benefits. 
We can see some paintings that we did not have much opportunity to admire 
before.</p>
<p>It is a privilege for me to be here with all of you again. Many things have 
changed since the last time we were together, but I am sure that our collective 
desire to serve the best interests of Canadians remains intact.</p>
<p>[<i>English</i>]</p>
<p>We have a lot of work in front of us. First, we have to continue our hard 
work toward the modernization of this institution to ensure a more efficient, 
transparent and accountable Senate that serves Canadians. I have no doubt that 
together we can succeed in our efforts to improve the perception that Canadians 
have of this institution.</p>
<p>[<i>Translation</i>]</p>
<p>Obviously we have to deal with the changes the new government intends to make 
to the w

<p>Our Prime Minister Trudeau attended the memorial for these tragic events. 
There he said, "It's a moment to remember and to make promises . . . ."</p>
<p>Today, I stand and hope we can make two promises in memory of the victims of 
those tragic attacks.</p>
<p>We now know the victims were targeted because they were women. The gunman was 
clear about this before taking his own life. He hated feminists and women's role 
in society. I am really troubled on this particular anniversary because it 
reminds me of how much work is yet to be done to elevate our own society to 
accept women as equal to men.</p>
<p>The women whose lives were taken chose to attend an engineering school, a 
place not typically seen as a place for women. Twenty-six years later, we see 
many stories of the rampant sexism that exists in the growing engineering 
sector.</p>
<p>In an age where technology is our future, we need to educate our society to 
accept and welcome strong women. Women have a place as leaders i

<p><b>The Hon. the Speaker:</b> Is leave granted, honourable senators?</p>
<p><b>The Hon. the Speaker:</b> Is leave granted, honourable senators?</p>
<p>[<i>Translation</i>]</p>
<p></p>
<p>[<i>English</i>]</p>
<p></p>
<p>(Bill read first time.)</p>
<p><b>The Hon. the Speaker:</b> Honourable senators, when shall this bill be 
read the second time?</p>
<p>(On motion of Senator Cowan, bill placed on the Orders of the Day for second 
reading two days hence.)</p>
<p>(Bill read first time.)</p>
<p><b>The Hon. the Speaker:</b> Honourable senators, when shall this bill be 
read the second time?</p>
<p>(On motion of Senator Cools, bill placed on the Orders of the Day for second 
reading two days hence.)</p>
<p>(Bill read first time.)</p>
<p><b>The Hon. the Speaker:</b> Honourable senators, when shall this bill be 
read the second time?</p>
<p>(On motion of Senator Moore, bill placed on the Orders of the Day for second 
reading two days hence.)</p>
<p>(Bill read first time.)</p>
<p><b>The Hon. t

adopt the motion?</p>
<p>(Motion agreed to.)</p>
<p>That the Standing Senate Committee on National Finance be authorized to 
	examine and report upon the expenditures set out in the Supplementary 
	Estimates (B) for the fiscal year ending March 31, 2016; and</p>
<p>That the committee be authorized to meet on Wednesday, December 9, 2015, 
	Thursday, December 10, 2015 and Friday, December 11, 2015, for the purposes 
	of its study of the expenditures set out in the Supplementary Estimates (B), 
	even though the Senate may then be sitting, with the application of rule 
	12-18(1) being suspended in relation thereto.</p>
<p><b>The Hon. the Speaker:</b> Is it your pleasure, honourable senators, to 
adopt the motion?</p>
<p>(Motion agreed to.)</p>
<p>The Senate proceeded to consideration of His Excellency the Governor 
General's Speech from the Throne at the opening of the First Session of the 
Forty-second Parliament.</p>
<p>That the following Address be presented to His Excellency the Govern

...

<p><b>The Hon. the Speaker</b> informed the Senate that a message had been 
received from the House of Commons with Bill C-3, An Act for granting to Her 
Majesty certain sums of money for the federal public administration for the 
financial year ending March 31, 2016.</p>
<p>(Bill read first time.)</p>
<p><b>The Hon. the Speaker:</b> Honourable senators, when shall this bill be 
read the second time?</p>
<p><b>The Hon. the Speaker:</b> Is leave granted, honourable senators?</p>
<p>(On motion of Senator Day, bill placed on the Orders of the Day for second 
reading later this day.)</p>
<p>(0910)</p>
<p>(Bill read first time.)</p>
<p><b>The Hon. the Speaker:</b> Honourable senators, when shall this bill be 
read the second time?</p>
<p>(On motion of Senator Dyck, bill placed on the Orders of the Day for second 
reading two days hence.)</p>
<p>That all committees be authorized to meet during any suspension of 
	today's sitting, with rule 12-18(1) being suspended in relation thereto; and</p

being self-serving.</p>
<p>I do not think this is a good way to run a parliament. It seems to me that 
there is something wrong with the system down there. When you write to the 
Speaker of the other place, Your Honour, I urge you to recall that this is not 
the first time we have received a defective bill — a seriously defective bill — 
and suggest that perhaps, in order to avoid these wildly inappropriate events, 
they examine their system and that perhaps they might even one day consider 
apologizing to us.</p>
<p><b>Some Hon. Senators:</b> Hear, hear.</p>
<p><b>The Hon. the Speaker:</b> Is leave granted, honourable senators?</p>
<p><b>Senator Campbell:</b> The question that I have —</p>
<p><b>The Hon. the Speaker:</b> Senator Campbell, is it on debate?</p>
<p><b>Senator Campbell:</b> On debate. I have a question, and I don't know who, 
quite frankly, can answer it, but my question is this: Was this schedule 
attached to this bill when it went through the house on December10? Can an

<p><b>Senator Cools:</b> I need less, usually.</p>
<p>When you hear<i> Messiah</i> in the next few days and you hear those words, 
this is where it comes from.</p>
<p>Peace, the Prince of Peace. Christmas and the Christ child herald rebirth, 
regeneration and renewal in the healing and redemptive power of love. 
Christianity is the promise of complete human redemption, through Jesus Christ, 
the new and eternal covenant.</p>
<p>Her Majesty's Canadian First Minister Trudeau and Her Canadian government's 
actions on behalf of these suffering, war-weary and war-torn displaced human 
beings are truly remarkable. This government's actions are outstanding and 
exceptional among human endeavours. I thank them. I praise them. I Praise God 
for this mercy to these suffering people.</p>
<p>To all of you, I say,<i> Gloria in excelsis deo</i>. Glory to God in the 
highest, and on earth peace to men. Merry Christmas.</p>
<p>Remember the Syrians. It was such a great and beautiful country. It 
is the

<p>(1220)</p>
<p>I'd like to thank the Finance Committee that worked on this over the last 
three days, and to thank all of you for your support of the Finance Committee 
over the years. Merry Christmas.</p>
<p><b>The Hon. the Speaker:</b> Are honourable senators ready for the question?</p>
<p><b>The Hon. the Speaker:</b> It was moved by the Honourable Senator Day, 
seconded by the Honourable Senator Mercer, that the bill be read the third time.</p>
<p>Is it your pleasure, honourable senators, to adopt the motion?</p>
<p><b>Some Hon. Senators:</b> Agreed.</p>
<p><b>Some Hon. Senators:</b> On division.</p>
<p>(Motion agreed to and bill read third time and passed, on division.)</p>
<p>[<i>Translation</i>]</p>
<p></p>
<p><b>The Hon. the Speaker</b> informed the Senate that the following 
communication had been received:</p>
<p>Mr. Speaker,</p>
<p>I have the honour to inform you that the Right Honourable David Johnston, 
	Governor General of Canada, will proceed to the Senate Chamber today

KeyboardInterrupt: 